# Linearized Optimization for Heat Pumps with Buffer Tank - Assign additional costs according to buffer tank temperature - Calculate costs for the complete year
See article for further details.

Dispatch optimization of heat pump and buffer tank calculating buffer tank temperature, but the CoP depends only on the pre calculated flow temperature.

Copyright, 2024, Mathias Moog, Hochschule Ansbach, Deutschland, CC-BY-NC-SA


## Load Modules, Data and Basic Definitions

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy import optimize
import time
# Import everythin vrom BasicDefintions.py
from BasicDefinitions import *

## Load a complete year and overwrite the global variables


In [2]:
# Read Data file - one year
df = pd.read_csv( "YearHourly.csv", header=None, index_col=0)
df.columns=["Tout","Qhouse","Price el."]
# Vergabe eines Names für die Index Spalte
df.index.name="Time"
# Zeige die ersten Zeilen an
#df.head()
# Show column and data type information
df.info()
# Overview plot
#df.plot()

<class 'pandas.core.frame.DataFrame'>
Index: 8760 entries, 2021-01-01 00:00:00 to 2021-12-31 23:00:00
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Tout       8760 non-null   float64
 1   Qhouse     8760 non-null   float64
 2   Price el.  8760 non-null   float64
dtypes: float64(3)
memory usage: 273.8+ KB


In [3]:
Tout = df["Tout"].to_numpy()
Tflow = fTflow(Tout)
Qhouse = df["Qhouse"].to_numpy()
p = df["Price el."].to_numpy()
n = p.size

## Set up the linear problem using numyp

It looks a bit more complicated than the Octave code since lots of mathematical operations are needed and the bounds data structure (List of sequences) is not direct compatible with numpy arrays.

In [1]:
# Elemenatary matrices, like in Octave BasicDefinitions.m
E = np.eye(n)
Z = np.zeros_like(E)
S = np.tril(np.ones_like(E),0)
# Matrices for the linear problem, see report
# In numpy matmul or dot are needed, the * is not the matrix multiplication from linear algebra!
M = S/C 
r = Tinit - np.matmul(S,Qhouse)/C
# Equality constraint for calculating Tbuffer
AE = np.hstack((-M, E)) 
bE = r
# Produce enough energy
# Invert sign for lower bound , there's no lower bound A x >= b, only an upper bount A x <= b
A_up = -np.hstack( (np.ones(shape=(1,n)),np.zeros(shape=(1,n)) ) )
b_up = -Qhouse.sum()
# cost vector 
a=3
cel = p / CoP(Tout,Tflow)
cte = p/CoP(Tout,Tflow+1)-p/CoP(Tout,Tflow)
c = np.hstack( (cel,a*cte))
# Bounds, x in [0,1], Tbuffer in [Tflow,Tmax]
bounds = [(0,Pth)]*n +  [ (Tflow[i].item(),Tmax) for i in range(n)] 


NameError: name 'np' is not defined

In [2]:
# Reference Costs, demand driven solution
print(f"Costs demand driven solution {cel.dot(Qhouse):.2f} €")

NameError: name 'cel' is not defined

## Run linear solver
Use scipy.optimize.linprog 

Be aware, that this linear solver takes equality constraints and upper limits separately. Lower limits are not supported! See above trick: Multiply with -1

In [11]:
start = time.time() # start time measurement
# run solver
opt = optimize.linprog(c=c, A_ub=A_up, b_ub=b_up, A_eq=AE, b_eq=bE, bounds=bounds)
end = time.time() # stop time measurement
# Print runtime
print(f"Runtime linear solver { end - start:.2f} s")
# Extract data for later use
Qprod = opt.x[0:n]
# Summary
print(f"Costs including temperature costs {opt.fun:.2f} €")
print(f"Costs excluding temperature costs {cel.dot(Qprod):.2f} €")
# show complete result
opt

Runtime linear solver 528.30 s
Costs including temperature costs 1798.33 €
Costs excluding temperature costs 705.31 €


        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 1798.3271310114521
              x: [ 1.113e+00  5.375e-02 ...  2.989e+01  3.000e+01]
            nit: 9949
          lower:  residual: [ 1.113e+00  5.375e-02 ...  0.000e+00
                              2.109e-01]
                 marginals: [ 0.000e+00  0.000e+00 ...  6.657e-03
                              0.000e+00]
          upper:  residual: [ 1.089e+01  1.195e+01 ...  3.011e+01
                              3.000e+01]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
                 marginals: [ 8.412e-06  1.174e-03 ... -6.352e-04
                              6.034e-03]
        ineqlin:  residual: [ 0.000e+00]
                 marginals: [-5.623e-02]
 mip_node_count: 0
 mip_dual_

In [7]:
opt


       message: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is None)
       success: False
        status: 2
           fun: None
             x: None
           nit: 0
         lower:  residual: None
                marginals: None
         upper:  residual: None
                marginals: None
         eqlin:  residual: None
                marginals: None
       ineqlin:  residual: None
                marginals: None